In [3]:
import numpy as np
import pandas as pd
import psycopg2
import getpass
# for configuring connection 
from configobj import ConfigObj
import os


In [3]:
# Create a database connection using settings from config file
config='../db/config.ini'

# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = ''
    conn_info["sqlhost"] = '192.168.60.144'
    conn_info["sqlport"] = 6432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu_crd'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == '192.168.60.144') & (conn_info["sqlport"]=='6432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres


In [4]:
from sqlalchemy import create_engine,text
con= create_engine('postgresql://eicu@192.168.60.144:6432/eicu')

In [4]:
datadir = '/home/mei/nas/docker/dataset/EICU/eicu-collaborative-research-database-2.0/'
csv = '/home/mei/nas/docker/thesis/data/csv/'

## flat_feature table: 
- select patient who comes from direct and emergency and the stay length in icu between 1 and 14 days

In [36]:
create_table_query =  query_schema +"""

drop materialized view if exists flat cascade;
create materialized view flat as
SELECT DISTINCT patientunitstayid, gender, age, apacheadmissiondx, admissionweight,dischargeweight, unitdischargelocation,unitdischargestatus
FROM patient
WHERE unitadmitsource IN ('Emergency Department', 'Direct Admit') 
    AND unitdischargeoffset >=1440 
    AND unitdischargeoffset <=20160
group by patientunitstayid
order by patientunitstayid
"""

In [37]:
with con.begin() as connection:
    connection.execute(text(create_table_query))
    
select_query = "SELECT * FROM flat;"
df_flat = pd.read_sql_query(select_query, con)

In [38]:
df_flat

,patientunitstayid,gender,age,apacheadmissiondx,admissionweight,dischargeweight,unitdischargelocation,unitdischargestatus
0,141168,Female,70,"Rhythm disturbance (atrial, supraventricular)",84.30,85.8,Death,Expired
1,141265,Male,67,"CVA, cerebrovascular accident/stroke",100.00,91.8,Floor,Alive
2,141266,Male,73,"Sepsis, renal/UTI (including bladder)",120.40,112.9,Floor,Alive
3,141276,Female,59,"Arrest, respiratory (without cardiac arrest)",156.60,156.6,Home,Alive
4,141284,Male,63,Anemia,NaN,88.5,Floor,Alive
...,...,...,...,...,...,...,...,...
68441,3353147,Male,24,"Overdose, sedatives, hypnotics, antipsychotics...",153.40,151.3,Floor,Alive
68442,3353194,Female,51,Cardiac arrest (with or without respiratory ar...,63.05,65.8,Death,Expired
68443,3353213,Female,51,Coma/change in level of consciousness (for hep...,54.40,59.4,Home,Alive
68444,3353226,Female,79,"Effusions, pleural",58.40,55.1,Death,Expired


##  labels
- unitdischarge location, unitdiscahrges status, actualiculos

In [103]:
create_table_query = query_schema + """
DROP MATERIALIZED VIEW  IF EXISTS labels CASCADE;
CREATE MATERIALIZED VIEW labels AS
SELECT DISTINCT ON (f.patientunitstayid) 
    f.patientunitstayid,
    f.dischargeweight,
    f.unitdischargelocation,
    f.unitdischargestatus,
    apr.actualiculos
FROM flat AS f
INNER JOIN apachepatientresult AS apr 
ON f.patientunitstayid = apr.patientunitstayid;
"""

In [104]:
with con.begin() as connection:
    connection.execute(text(create_table_query)) 
select_query = "SELECT * FROM labels;"
df_labels = pd.read_sql_query(select_query, con)

In [105]:
df_labels

,patientunitstayid,dischargeweight,unitdischargelocation,unitdischargestatus,actualiculos
0,141168,85.8,Death,Expired,2.4972
1,141265,91.8,Floor,Alive,4.2138
2,141266,112.9,Floor,Alive,1.0423
3,141276,156.6,Home,Alive,1.1694
4,141284,88.5,Floor,Alive,1.4416
...,...,...,...,...,...
60330,3353147,151.3,Floor,Alive,1.0888
60331,3353194,65.8,Death,Expired,2.4930
60332,3353213,59.4,Home,Alive,3.4118
60333,3353226,55.1,Death,Expired,8.0368


## diagnosis

In [110]:
create_table_query =  query_schema +"""
drop materialized view if exists diagnoses cascade;
create materialized view diagnoses as

  -- for past medical history:
  select ph.patientunitstayid, ph.pasthistorypath as diagnosisstring
    from pasthistory as ph
    inner join labels as l on l.patientunitstayid = ph.patientunitstayid
    where ph.pasthistoryoffset > 0
    and ph.pasthistorypath LIKE '%Organ Systems%'
       
"""

In [111]:
with con.begin() as connection:
    connection.execute(text(create_table_query)) 
    
select_query = "SELECT * FROM diagnoses;"
df_dx = pd.read_sql_query(select_query, con)

In [112]:
df_dx

,patientunitstayid,diagnosisstring
0,141168,notes/Progress Notes/Past History/Organ System...
1,141168,notes/Progress Notes/Past History/Organ System...
2,141168,notes/Progress Notes/Past History/Organ System...
3,141168,notes/Progress Notes/Past History/Organ System...
4,141168,notes/Progress Notes/Past History/Organ System...
...,...,...
279753,3353251,notes/Progress Notes/Past History/Organ System...
279754,3353251,notes/Progress Notes/Past History/Organ System...
279755,3353251,notes/Progress Notes/Past History/Organ System...
279756,3353251,notes/Progress Notes/Past History/Organ System...


## medication

In [113]:
query = query_schema + """
select a.patientunitstayid, a.drugname
from admissiondrug a
inner join labels l on a.patientunitstayid = l.patientunitstayid
where a.drugoffset > 0
order by a.patientunitstayid
"""

In [114]:
df_drug = pd.read_sql_query(query,con)

In [115]:
df_drug

,patientunitstayid,drugname
0,252784,MULTIVITAMIN ...
1,252784,NITROGLYCERIN ...
2,252784,LISINOPRIL ...
3,252784,LANTUS ...
4,252784,CHOLECALCIFEROL (VITAMIN D3) ...
...,...,...
285418,3348105,METFORMIN HCL ...
285419,3348105,TRIAMTERENE W/HCTZ ...
285420,3348105,DILTIAZEM HCL ...
285421,3348105,TRIAMTERENE W/HCTZ ...


## lab_ ts

In [6]:
create_table_query =  query_schema +"""
drop materialized view if exists timeserieslab cascade;
create materialized view timeserieslab as
  select l.patientunitstayid, l.labresultoffset, l.labname,l.labresult
    from lab as l
    inner join labels as la
      on la.patientunitstayid = l.patientunitstayid -- only extract data for the cohort
    where l.labresultoffset > 0;

"""

In [21]:
with con.begin() as connection:
    connection.execute(text(create_table_query)) 

select_query = "SELECT * FROM timeserieslab;"
df_timeserieslab = pd.read_sql_query(select_query, con)

In [22]:
df_timeserieslab

,patientunitstayid,labresultoffset,labname,labresult
0,141168,2026,fibrinogen,177.000
1,141168,1133,PT - INR,2.500
2,141168,2026,magnesium,2.000
3,141168,1133,PT,26.600
4,141168,2141,pH,7.200
...,...,...,...,...
10434833,3353251,4049,bedside glucose,129.000
10434834,3353251,1849,BUN,32.000
10434835,3353251,310,pH,7.194
10434836,3353251,409,potassium,4.400


In [9]:
min_offset = df_timeserieslab["labresultoffset"].min()
max_offset = df_timeserieslab["labresultoffset"].max()

print(f"最小 labresultoffset: {min_offset}")
print(f"最大 labresultoffset: {max_offset}")

# 找到最小和最大的 labresultoffset 对应的索引
min_index = df_timeserieslab["labresultoffset"].idxmin()
max_index = df_timeserieslab["labresultoffset"].idxmax()

# 打印对应的行
print("最小 labresultoffset 对应的行：")
print(df_timeserieslab.loc[min_index])

print("\n最大 labresultoffset 对应的行：")
print(df_timeserieslab.loc[max_index])

最小 labresultoffset: 1
最大 labresultoffset: 526696
最小 labresultoffset 对应的行：
patientunitstayid             143471
labresultoffset                    1
labname              bedside glucose
labresult                      123.0
Name: 12470, dtype: object

最大 labresultoffset 对应的行：
patientunitstayid    2758367
labresultoffset       526696
labname              lactate
labresult                0.8
Name: 7643877, dtype: object


## vital periodic

In [123]:
create_table_query =  query_schema +"""
drop materialized view if exists timeseriesperiodic cascade;
create materialized view timeseriesperiodic as
  select vp.patientunitstayid, vp.observationoffset, vp.temperature, vp.sao2, vp.heartrate, vp.respiration, vp.cvp,
    vp.systemicsystolic, vp.systemicdiastolic, vp.systemicmean, vp.st1, vp.st2, vp.st3
    from vitalperiodic as vp
    -- select only the patients who are in the cohort
    inner join labels as la
      on la.patientunitstayid = vp.patientunitstayid
    where vp.observationoffset > 0
    order by vp.patientunitstayid, vp.observationoffset;
"""

In [125]:
with con.begin() as connection:
    connection.execute(text(create_table_query)) 

select_query = "SELECT * FROM timeseriesperiodic;"
df_timeseriesperiodic = pd.read_sql_query(select_query, con)

In [127]:
min = df_timeseriesperiodic["observationoffset"].min()
max = df_timeseriesperiodic["observationoffset"].max()

print(f"最小 observationoffset: {min_offset}")
print(f"最大 observationoffset {max_offset}")


最小 observationoffset: 1
最大 observationoffset 526696


In [126]:
df_timeseriesperiodic

,patientunitstayid,observationoffset,temperature,sao2,heartrate,respiration,cvp,systemicsystolic,systemicdiastolic,systemicmean,st1,st2,st3
0,141168,119,NaN,93.0,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141168,124,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,141168,129,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,141168,134,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,141168,139,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285007,3353251,16219,NaN,84.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51285008,3353251,16224,NaN,68.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51285009,3353251,16229,NaN,82.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51285010,3353251,16234,NaN,95.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# df_dx=pd.read_csv(csv+'/diagnoses.csv')
# df_timeserieslab=pd.read_csv(csv+'/timeserieslab.csv')
# df_timeseriesperiodic=pd.read_csv(csv+'/timeseriesperiodic.csv')
# df_labels=pd.read_csv(csv+'/labels.csv')
# df_flat=pd.read_csv(csv+'/flat.csv')


In [128]:
common_ids = set(df_labels['patientunitstayid'].unique()) \
             .intersection(df_dx['patientunitstayid'].unique()) \
             .intersection(df_timeserieslab['patientunitstayid'].unique()) \
             .intersection(df_timeseriesperiodic['patientunitstayid'].unique()) \
             .intersection(df_flat['patientunitstayid'].unique())\
             .intersection(df_drug['patientunitstayid'].unique())

In [130]:
common_ids_df = pd.DataFrame(list(common_ids), columns=['patientunitstayid'])
common_ids_df.to_csv(csv+'/common_ids.csv', index=False)

In [23]:
common_ids=pd.read_csv(csv+'/common_ids.csv')

In [11]:
len(common_ids)

12260

In [13]:
def filter_and_sort(dataframes, common_ids, id_column='patientunitstayid'):
    """
    Filter and sort dataframes by common
    """
    return [
        df[df[id_column].isin(common_ids)].sort_values(by=id_column)
        for df in dataframes
    ]

In [132]:
dataframes = [df_labels, df_dx, df_timeserieslab, df_timeseriesperiodic, df_flat, df_drug]
filtered_sorted_dataframes = filter_and_sort(dataframes, common_ids)

# back to the original dataframes
df_labels, df_dx, df_timeserieslab, df_timeseriesperiodic, df_flat,df_drug = filtered_sorted_dataframes

In [36]:
def export_table_to_csv(df, output_file):
    """
    Export a SQL query to a CSV file
    """
    df.to_csv(output_file, index=False)
    print(f"Exported {output_file}")


In [134]:
## labels
export_table_to_csv(df_labels, f"{csv}labels.csv")
## static data
export_table_to_csv(df_dx, f"{csv}diagnoses.csv")
export_table_to_csv(df_flat, f"{csv}flat.csv")
export_table_to_csv(df_drug, f"{csv}drug.csv")

## time series data
export_table_to_csv(df_timeserieslab, f"{csv}timeserieslab.csv")
export_table_to_csv(df_timeseriesperiodic, f"{csv}timeseriesperiodic.csv")


Exported /home/mei/nas/docker/thesis/data/csv/labels.csv
Exported /home/mei/nas/docker/thesis/data/csv/diagnoses.csv
Exported /home/mei/nas/docker/thesis/data/csv/flat.csv
Exported /home/mei/nas/docker/thesis/data/csv/drug.csv
Exported /home/mei/nas/docker/thesis/data/csv/timeserieslab.csv
Exported /home/mei/nas/docker/thesis/data/csv/timeseriesperiodic.csv


In [135]:
df_flat

,patientunitstayid,gender,age,apacheadmissiondx,admissionweight,dischargeweight,unitdischargelocation,unitdischargestatus
4177,252784,Male,56,Diabetic ketoacidosis,75.00,75.7,Floor,Alive
4189,253331,Male,76,"CHF, congestive heart failure",114.70,114.4,Floor,Alive
4212,255112,Male,52,Cardiac arrest (with or without respiratory ar...,50.60,58.2,Death,Expired
4280,258354,Female,61,"Hypertension, uncontrolled (for cerebrovascula...",74.38,75.3,Floor,Alive
4300,259414,Male,81,"CVA, cerebrovascular accident/stroke",107.50,106.7,Floor,Alive
...,...,...,...,...,...,...,...,...
66723,3247360,Female,25,"Sepsis, renal/UTI (including bladder)",48.90,60.3,Floor,Alive
66726,3247421,Male,59,"Sepsis, pulmonary",76.20,82.1,Floor,Alive
68010,3346588,Female,71,"CHF, congestive heart failure",121.50,115.3,Telemetry,Alive
68073,3347496,Female,56,"Overdose, sedatives, hypnotics, antipsychotics...",71.20,71.3,Floor,Alive


In [136]:
df_labels

,patientunitstayid,dischargeweight,unitdischargelocation,unitdischargestatus,actualiculos
4005,252784,75.7,Floor,Alive,2.0500
4016,253331,114.4,Floor,Alive,1.7625
4037,255112,58.2,Death,Expired,10.7381
4102,258354,75.3,Floor,Alive,3.0090
4122,259414,106.7,Floor,Alive,5.8562
...,...,...,...,...,...
58668,3247360,60.3,Floor,Alive,3.0097
58671,3247421,82.1,Floor,Alive,2.9520
59916,3346588,115.3,Telemetry,Alive,2.1826
59978,3347496,71.3,Floor,Alive,1.5340


In [137]:
df_dx

,patientunitstayid,diagnosisstring
10195,252784,notes/Progress Notes/Past History/Organ System...
10194,252784,notes/Progress Notes/Past History/Organ System...
10193,252784,notes/Progress Notes/Past History/Organ System...
10335,253331,notes/Progress Notes/Past History/Organ System...
10340,253331,notes/Progress Notes/Past History/Organ System...
...,...,...
278651,3348105,notes/Progress Notes/Past History/Organ System...
278650,3348105,notes/Progress Notes/Past History/Organ System...
278649,3348105,notes/Progress Notes/Past History/Organ System...
278655,3348105,notes/Progress Notes/Past History/Organ System...


In [138]:
df_drug

,patientunitstayid,drugname
0,252784,MULTIVITAMIN ...
10,252784,FLAX SEED OIL ...
9,252784,OMEPRAZOLE ...
8,252784,CARVEDILOL ...
7,252784,NOVOLOG ...
...,...,...
285404,3348105,PRAVASTATIN SODIUM ...
285403,3348105,OMEPRAZOLE ...
285402,3348105,RAMIPRIL ...
285411,3348105,METFORMIN HCL ...


In [32]:
df_timeserieslab

,patientunitstayid,labresultoffset,labname,labresult
0,252784,1080,bedside glucose,109.00
1,252784,1487,sodium,137.00
2,252784,1140,bedside glucose,97.00
3,252784,2865,pH,7.47
4,252784,2550,bedside glucose,93.00
...,...,...,...,...
2337782,3348105,2985,MCV,84.00
2337783,3348105,4493,MPV,7.80
2337784,3348105,1550,BUN,52.00
2337785,3348105,4493,platelets x 1000,275.00


In [140]:
df_timeseriesperiodic

,patientunitstayid,observationoffset,temperature,sao2,heartrate,respiration,cvp,systemicsystolic,systemicdiastolic,systemicmean,st1,st2,st3
3252945,252784,10,NaN,100.0,106.0,22.0,NaN,NaN,NaN,NaN,0.05,0.05,0.0
3253330,252784,1940,NaN,97.0,93.0,21.0,NaN,NaN,NaN,NaN,0.00,0.00,0.0
3253331,252784,1945,NaN,98.0,92.0,21.0,NaN,NaN,NaN,NaN,0.00,0.00,0.0
3253332,252784,1950,NaN,98.0,90.0,21.0,NaN,NaN,NaN,NaN,0.00,0.00,0.0
3253333,252784,1955,NaN,98.0,87.0,20.0,NaN,NaN,NaN,NaN,0.00,0.10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51005542,3348105,1428,NaN,100.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51005543,3348105,1433,NaN,100.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51005544,3348105,1438,NaN,100.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51005546,3348105,1448,NaN,100.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## get the common ids in all the processed tables

In [1]:
hdf= '/home/mei/nas/docker/thesis/data/hdf/'

In [22]:
diagnoses = pd.read_csv(csv+'/preprocessed_diagnoses.csv')
flat_drug = pd.read_csv(csv+'/preprocessed_flat_drug.csv')
flat = pd.read_csv(csv+'/preprocessed_flat.csv')
labels = pd.read_csv(csv+'/preprocessed_labels.csv')
timeseries = pd.read_hdf(hdf+'/final_timeseries.h5').reset_index()

In [23]:
timeseries

,patient,time,-bands,-basos,-eos,-lymphs,-monos,-polys,24 h urine protein,24 h urine urea nitrogen,...,sao2,heartrate,respiration,cvp,systemicsystolic,systemicdiastolic,systemicmean,st1,st2,st3
0,252784,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100.0,106.0,22.0,0.0,0.0,0.0,0.0,0.05,0.05,0.0
1,252784,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100.0,107.0,24.0,0.0,0.0,0.0,0.0,0.10,0.00,-0.1
2,252784,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100.0,109.0,21.0,0.0,0.0,0.0,0.0,0.00,0.10,0.0
3,252784,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100.0,110.0,26.0,0.0,0.0,0.0,0.0,0.10,0.00,-0.1
4,252784,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,98.0,116.0,26.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18094136,3348105,1500,0.0,1.0,2.0,12.0,13.0,0.0,0.0,0.0,...,98.0,113.0,9.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
18094137,3348105,1501,0.0,1.0,2.0,12.0,13.0,0.0,0.0,0.0,...,98.0,113.0,9.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
18094138,3348105,1502,0.0,1.0,2.0,12.0,13.0,0.0,0.0,0.0,...,98.0,113.0,9.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
18094139,3348105,1503,0.0,1.0,2.0,12.0,13.0,0.0,0.0,0.0,...,98.0,113.0,9.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0


In [24]:
common_ids = set(labels['patient'].unique())\
               .intersection(diagnoses['patient'].unique())\
                .intersection(flat_drug['patient'].unique())\
                .intersection(flat['patient'].unique())\
                .intersection(timeseries['patient'].unique())

In [20]:
## save common id
common_ids= pd.DataFrame(list(common_ids), columns=['patient'])
common_ids.to_csv(csv+'/common_ids.csv', index=False)

In [25]:
len(common_ids)

11698

In [27]:
def filter_and_sort_patient(dataframes, common_ids, id_column='patient'):
    """
    Filter and sort dataframes by common
    """
    return [
        df[df[id_column].isin(common_ids)].sort_values(by=id_column)
        for df in dataframes
    ]

In [28]:
dataframes = [labels, diagnoses, flat_drug, flat, timeseries]
filtered_sorted_dataframes = filter_and_sort_patient(dataframes, common_ids)

# back to the original dataframes
labels, diagnoses, flat_drug, flat, timeseries= filtered_sorted_dataframes

In [37]:
## labels
export_table_to_csv(labels, f"{csv}final_labels.csv")

## static data
export_table_to_csv(diagnoses, f"{csv}final_diagnoses.csv")
export_table_to_csv(flat_drug, f"{csv}final_flat_drug.csv")
export_table_to_csv(flat, f"{csv}final_flat.csv")


Exported /home/mei/nas/docker/thesis/data/csv/final_labels.csv
Exported /home/mei/nas/docker/thesis/data/csv/final_diagnoses.csv
Exported /home/mei/nas/docker/thesis/data/csv/final_flat_drug.csv
Exported /home/mei/nas/docker/thesis/data/csv/final_flat.csv


In [49]:
# time series data
timeseries.to_hdf(hdf+'final_timeseries.h5', key='df', mode='w')

In [29]:
labels

,patient,dischargeweight,unitdischargestatus,actualiculos,discharge_risk_category
0,252784,75.7,0,2.0500,1
1,253331,114.4,0,1.7625,1
2,255112,58.2,1,10.7381,4
3,258354,75.3,0,3.0090,1
4,259414,106.7,0,5.8562,1
...,...,...,...,...,...
12137,3247116,68.0,1,8.1256,4
12139,3247421,82.1,0,2.9520,1
12140,3346588,115.3,0,2.1826,2
12141,3347496,71.3,0,1.5340,1


In [30]:
diagnoses

,patient,Cardiovascular (R),Cardiovascular (R)|AICD,Cardiovascular (R)|Angina,Cardiovascular (R)|Arrhythmias,Cardiovascular (R)|Arrhythmias|atrial fibrillation - chronic,Cardiovascular (R)|Arrhythmias|atrial fibrillation - intermittent,Cardiovascular (R)|Congestive Heart Failure,Cardiovascular (R)|Congestive Heart Failure|CHF,Cardiovascular (R)|Congestive Heart Failure|CHF - severity unknown,...,"apacheadmissiondx_Rhythm disturbance (atrial, supraventricular)",apacheadmissiondx_Rhythm disturbance (conduction defect),apacheadmissiondx_Seizures (primary-no structural brain disease),"apacheadmissiondx_Sepsis, GI","apacheadmissiondx_Sepsis, cutaneous/soft tissue","apacheadmissiondx_Sepsis, other","apacheadmissiondx_Sepsis, pulmonary","apacheadmissiondx_Sepsis, renal/UTI (including bladder)","apacheadmissiondx_Sepsis, unknown","grouped_apacheadmissiondx_Overdose,"
0,252784,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,253331,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,255112,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,258354,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,259414,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12254,3247116,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
12256,3247421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
12257,3346588,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
12258,3347496,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,1


In [31]:
flat_drug

,patient,gender,age,admissionweight,> 89,ACETAMINOPHEN,ADVAIR DISKUS,ALBUTEROL,ALBUTEROL SULFATE,ALLOPURINOL,...,TRAMADOL HCL,TRAZODONE HCL,TYLENOL,VITAMIN C,VITAMIN D,VITAMIN D3,WARFARIN SODIUM,XANAX,ZOCOR,ZOFRAN
0,252784,1,56.0,75.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,253331,1,76.0,114.7,0,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0
2,255112,1,52.0,50.6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,258354,0,61.0,74.4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,259414,1,81.0,107.5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11797,3247116,0,52.0,61.7,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
11798,3247421,1,59.0,76.2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11799,3346588,0,71.0,121.5,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
11800,3347496,0,56.0,71.2,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [34]:
timeseries

,patient,time,-bands,-basos,-eos,-lymphs,-monos,-polys,24 h urine protein,24 h urine urea nitrogen,...,sao2,heartrate,respiration,cvp,systemicsystolic,systemicdiastolic,systemicmean,st1,st2,st3
0,252784,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100.0,106.0,22.0,0.0,0.0,0.0,0.0,0.05,0.05,0.0
390,252784,391,0.0,0.2,0.3,12.3,12.1,0.0,0.0,0.0,...,98.0,87.0,20.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
391,252784,392,0.0,0.2,0.3,12.3,12.1,0.0,0.0,0.0,...,97.0,86.0,21.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
392,252784,393,0.0,0.2,0.3,12.3,12.1,0.0,0.0,0.0,...,98.0,88.0,21.0,0.0,0.0,0.0,0.0,0.00,0.10,0.1
393,252784,394,0.0,0.2,0.3,12.3,12.1,0.0,0.0,0.0,...,97.0,93.0,22.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18093132,3348105,496,0.0,1.0,2.0,14.0,10.0,0.0,0.0,0.0,...,98.0,92.0,27.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
18093131,3348105,495,0.0,1.0,2.0,14.0,10.0,0.0,0.0,0.0,...,99.0,84.0,20.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
18093130,3348105,494,0.0,1.0,2.0,14.0,10.0,0.0,0.0,0.0,...,99.0,82.0,4.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
18093139,3348105,503,0.0,1.0,2.0,14.0,10.0,0.0,0.0,0.0,...,98.0,84.0,16.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0


In [50]:
timeseries

-bands  -basos  -eos  -lymphs  -monos  -polys  \
patient time                                                  
252784  1        0.0     0.0   0.0      0.0     0.0     0.0   
        2        0.0     0.0   0.0      0.0     0.0     0.0   
        3        0.0     0.0   0.0      0.0     0.0     0.0   
        4        0.0     0.0   0.0      0.0     0.0     0.0   
        5        0.0     0.0   0.0      0.0     0.0     0.0   
...              ...     ...   ...      ...     ...     ...   
3348105 1500     0.0     1.0   2.0     12.0    13.0     0.0   
        1501     0.0     1.0   2.0     12.0    13.0     0.0   
        1502     0.0     1.0   2.0     12.0    13.0     0.0   
        1503     0.0     1.0   2.0     12.0    13.0     0.0   
        1504     0.0     1.0   2.0     12.0    13.0     0.0   

              24 h urine protein  24 h urine urea nitrogen  ALT (SGPT)  \
patient time                                                             
252784  1                    0.0                       0.0         0.0   
        2                    0.0                       0.0         0.0   
        3                    0.0                       0.0         0.0   
        4                    0.0                       0.0         0.0   
        5                    0.0                       0.0         0.0   
...                          ...                       ...         ...   
3348105 1500                 0.0                       0.0        17.0   
        1501                 0.0                       0.0        17.0   
        1502                 0.0                       0.0        17.0   
        1503                 0.0                       0.0        17.0   
        1504                 0.0                       0.0        17.0   

              ANF/ANA  ...   sao2  heartrate  respiration  cvp  \
patient time           ...                                       
252784  1         0.0  ...  100.0      106.0         22.0  0.0   
        2         0.0  ...  100.0      107.0         24.0  0.0   
        3         0.0  ...  100.0      109.0         21.0  0.0   
        4         0.0  ...  100.0      110.0         26.0  0.0   
        5         0.0  ...   98.0      116.0         26.0  0.0   
...               ...  ...    ...        ...          ...  ...   
3348105 1500      0.0  ...   98.0      113.0          9.0  0.0   
        1501      0.0  ...   98.0      113.0          9.0  0.0   
        1502      0.0  ...   98.0      113.0          9.0  0.0   
        1503      0.0  ...   98.0      113.0          9.0  0.0   
        1504      0.0  ...   98.0      113.0          9.0  0.0   

              systemicsystolic  systemicdiastolic  systemicmean   st1   st2  \
patient time                                                                  
252784  1                  0.0                0.0           0.0  0.05  0.05   
        2                  0.0                0.0           0.0  0.10  0.00   
        3                  0.0                0.0           0.0  0.00  0.10   
        4                  0.0                0.0           0.0  0.10  0.00   
        5                  0.0                0.0           0.0  0.00  0.00   
...                        ...                ...           ...   ...   ...   
3348105 1500               0.0                0.0           0.0  0.00  0.00   
        1501               0.0                0.0           0.0  0.00  0.00   
        1502               0.0                0.0           0.0  0.00  0.00   
        1503               0.0                0.0           0.0  0.00  0.00   
        1504               0.0                0.0           0.0  0.00  0.00   

              st3  
patient time       
252784  1     0.0  
        2    -0.1  
        3     0.0  
        4    -0.1  
        5     0.0  
...           ...  
3348105 1500  0.0  
        1501  0.0  
        1502  0.0  
        1503  0.0  
        1504  0.0  

[18094141 rows x 162 columns]

In [33]:
flat

,patient,gender,age,admissionweight,> 89
0,252784,1,56.0,75.0,0
1,253331,1,76.0,114.7,0
2,255112,1,52.0,50.6,0
3,258354,0,61.0,74.4,0
4,259414,1,81.0,107.5,0
...,...,...,...,...,...
12228,3247116,0,52.0,61.7,0
12230,3247421,1,59.0,76.2,0
12231,3346588,0,71.0,121.5,0
12232,3347496,0,56.0,71.2,0
